# Federated Horizontal XGBoost with Tree-based Collaboration 

This tutorial illustrates a federated horizontal xgboost learning on tabular data with bagging collaboration. 

Before do the training, we need to setup NVFLARE

## Setup NVFLARE

Follow [Getting Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to set up a virtual environment and install NVFLARE.

You can also follow this [notebook](https://github.com/NVIDIA/NVFlare/blob/main/examples/nvflare_setup.ipynb) to get set up.

> Make sure you have installed nvflare from **terminal** 


## Install requirements
assuming the current directory is '/examples/hello-world/step-by-step/higgs/xgboost'

In [ ]:
!pwd

In [ ]:
%pip install -r requirements.txt

>Note:
In the upcoming sections, we'll utilize the 'tree' command. To install this command on a Linux system, you can use the sudo apt install tree command. As an alternative to 'tree', you can use the ls -al command.



## Prepare data
Please reference [prepare_higgs_data](../prepare_data.ipynb) notebooks. Pay attention to the current location. You need to switch "higgs" directory to run the data split.
    

Now we have our data prepared. we are ready to do the training

### Data Cleaning 

We noticed from time-to-time the Higgs dataset is making small changes which causing job to fail. so we need to do some clean up or skip certain rows. 
For example: certain floating number mistakenly add an alphabetical letter at some point of time. This may have already fixed by UCI. 


## XGBoost
This tutorial uses [XGBoost](https://github.com/dmlc/xgboost), which is an optimized distributed gradient boosting library.

### Federated XGBoost Model
Here we use tree-based collaboration for horizontal federated XGBoost.

Under this setting, individual trees are independently trained on each client's local data without aggregating the global sample gradient histogram information.
Trained trees are collected and passed to the server / other clients for bagging aggregation and further boosting rounds.

The XGBoost Booster api is leveraged to create in-memory Booster objects that persist across rounds to cache predictions from trees added in previous rounds and retain other data structures needed for training.

Let's look at the code see how we convert the local training script to the federated training script.

In [ ]:
!pwd

In [ ]:
!cat code/xgboost_fl.py

The code is pretty much like the local xgboost training script of `code/xgboost_local_oneshot.py`, and its derivative `code/xgboost_local_iter.py`.

#### load data

We first load the features from the header file: 
    
```
    site_name = flare.get_site_name()
    feature_data_path = f"{data_root_dir}/{site_name}_header.csv"
    features = load_features(feature_data_path)
    n_features = len(features) -1

    data_path = f"{data_root_dir}/{site_name}.csv"
    data = load_data(data_path=data_path, data_features=features, test_size=test_size, skip_rows=skip_rows)

```

then load the data from the main csv file, then transform the data and split the training and test data based on the test_size provided.  

```
    data = to_dataset_tuple(data)
    dataset = transform_data(data)
    x_train, y_train, train_size = dataset["train"]
    x_test, y_test, test_size = dataset["test"]

```

The part that's specific to Federated Learning is in the following codes

```
# (1) import nvflare client API
from nvflare import client as flare

```
```
# (2) initializes NVFlare client API
    flare.init()

    site_name = flare.get_site_name()
    
```
    
These few lines, import NVFLARE Client API and initialize it, then use the API to find the site_name (such as site-1, site-2 etc.). With the site-name, we can construct the site-specific 
data path such as

```
    feature_data_path = f"{data_root_dir}/{site_name}_header.csv"

    data_path = f"{data_root_dir}/{site_name}.csv"
```

#### Training 

In the standard traditional xgboost, we would train the model such as
```
  model = xgb.train(...) 
```

with federated learning, using FLARE Client API, we need to make a few changes
* 1) we are not only training in local iterations, but also global rounds, we need to keep the program running until we reached to the totoal number of rounds 
  
  ```
      while flare.is_running():
          ... rest of code
  
  ```
  
* 2) Unlike local learning, we have now have more than one clients/sites participating the training. To ensure every site starts with the same model parameters, we use server to broadcast the initial model parameters to every sites at the first round ( current_round = 0). 

* 3) We will need to use FLARE client API to receive global model updates 

```
        # (3) receives FLModel from NVFlare
        input_model = flare.receive()
        global_params = input_model.params
        curr_round = input_model.current_round
```

```
        if curr_round == 0:
            # (4) first round, no global model
            model = xgb.train(
                xgb_params,
                dmat_train,
                num_boost_round=1,
                evals=[(dmat_train, "train"), (dmat_test, "test")],
            )
            config = model.save_config()
        ....
```
* 4) if it is not the first round, we need to use the global model to update the local model before training the next round. 

```
            # (5) update model based on global updates
            model_updates = global_params["model_data"]
            for update in model_updates:
                global_model_as_dict = update_model(global_model_as_dict, json.loads(update))
            loadable_model = bytearray(json.dumps(global_model_as_dict), "utf-8")
            # load model
            model.load_model(loadable_model)
            model.load_config(config)
```

* 5) we then evaluate the global model using the local data

```
            # (6) evaluate model
            auc = evaluate_model(x_test, model, y_test)
```
* 6) finally we do the training 

```
            # train model in two steps
            # first, eval on train and test
            eval_results = model.eval_set(
                evals=[(dmat_train, "train"), (dmat_test, "test")], iteration=model.num_boosted_rounds() - 1
            )
            print(eval_results)
            # second, train for one round
            model.update(dmat_train, model.num_boosted_rounds())
        
```

* 7) we need the new training result (new tree) back to server for aggregation, to do that, we have the following code

```
        # (7) construct trained FL model
        # Extract newly added tree using xgboost_bagging slicing api
        bst_new = model[model.num_boosted_rounds() - 1 : model.num_boosted_rounds()]
        local_model_update = bst_new.save_raw("json")
        params = {"model_data": local_model_update}
        metrics = {"accuracy": auc}
        output_model = flare.FLModel(params=params, metrics=metrics)

        # (8) send model back to NVFlare
        flare.send(output_model)
```

## Prepare Job  

Now, we have the code, we need to prepare job folder with configurations to run in NVFLARE. To do this, we can leveage the job template for scikit learn. First look at the the available job templates

In [ ]:
!nvflare config -jt ../../../../../job_templates/

In [ ]:
!nvflare job list_templates

In [ ]:
!nvflare job create -j /tmp/nvflare/jobs/xgboost -force -w xgboost_tree \
-sd code \
-f config_fed_client.conf app_script="xgboost_fl.py" app_config="--data_root_dir /tmp/nvflare/dataset/output"

In [ ]:
!cat /tmp/nvflare/jobs/xgboost/app/config/config_fed_client.conf

In [ ]:
!tree /tmp/nvflare/jobs/xgboost

>Note 
 For potential on-the-fly data cleaning, we use skip_rows = 0 to skip 1st row. We could skip_rows = [0, 3] to skip first and 4th rows.



## Run job in simulator

We use the simulator to run this job

In [ ]:
!nvflare simulator /tmp/nvflare/jobs/xgboost -w /tmp/nvflare/xgboost -n 3 -t 3

Let's examine the results.

We can notice from the FL training log, at the last round (`current_round=99`) of local training, site-1 reports `site-1: global model AUC: 0.82085`
Under tree-based collaboration, the newly boosted trees from all clients will be bagged together to be added to the local models, hence, at the end of our 100-round training with 3 clients, the global model should have 300 trees in total. Let's double check if this holds:

In [ ]:
import json

# Specify the global model file path
file_path =  '/tmp/nvflare/xgboost/simulate_job/app_server/xgboost_model.json'

with open(file_path) as json_data:
    model = json.load(json_data)
    print(f"num_trees: {model['learner']['gradient_booster']['model']['gbtree_model_param']['num_trees']}")

And yes, we do have 300 trees for the global model as expected.

Now let's run some local trainings to verify if this number makes sense.

In [ ]:
!python3 ./code/xgboost_local_oneshot.py --data_root_dir /tmp/nvflare/dataset/output

In [ ]:
!python3 ./code/xgboost_local_iter.py --data_root_dir /tmp/nvflare/dataset/output

Both oneshot and iterative training schemes yield idential results of `local model AUC: 0.81928`. As compared with FL's `global model AUC: 0.82085`, we can notice FL gives some benefits, even under homogeneous data distribution across clients.

## We are done !
Congratulations! you have just completed the federated xgboost model for tabular data. 

You have now completed the HIGGS step-by-step example series.
Next either take a look at the [cifar10](../../cifar10/README.md) example series for how to train an image classifier with PyTorch, or explore the
[examples/advanced](../../../../advanced/README.md) directory for more in-depth examples.